Tools =>
Models can make requestthat perform tasks sucha s fetching from the database,serching the web or running the code. The toos are the pairing of :

1. A schema, including the name of the tool and the description of the tool or argument definitions (JSON schema)

2. A function that implements the tool


In [1]:
import os
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
# model=model = init_chat_model("groq:llama-3.1-8b-instant")
model=model = init_chat_model("groq:llama-3.3-70b-versatile")
response=model.invoke("What today's date")
response.content

"Today's date is January 19, 2024"

In [2]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])

In [3]:
response=model_with_tools.invoke("What is the weather in Delhi?")
print(response)
response.tool_calls

content='' additional_kwargs={'tool_calls': [{'id': 'sjsa0cmam', 'function': {'arguments': '{"location":"Delhi"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 219, 'total_tokens': 234, 'completion_time': 0.054582421, 'completion_tokens_details': None, 'prompt_time': 0.011037085, 'prompt_tokens_details': None, 'queue_time': 0.056601105, 'total_time': 0.065619506}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019bd6c3-a410-7e13-b807-21dbc87b6098-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Delhi'}, 'id': 'sjsa0cmam', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 219, 'output_tokens': 15, 'total_tokens': 234}


[{'name': 'get_weather',
  'args': {'location': 'Delhi'},
  'id': 'sjsa0cmam',
  'type': 'tool_call'}]

### Tool Execution Loop
-- For refernce see the image how actually work
-- Step 1: User input 
-- Step 2: Best tool selection and tool execution
-- Step 3: Result processing

In [4]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Banglore?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)    

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)

In [5]:
messages

[{'role': 'user', 'content': "What's the weather in Banglore?"},
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'vmrcvh9ph', 'function': {'arguments': '{"location":"Bangalore"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 220, 'total_tokens': 235, 'completion_time': 0.045525699, 'completion_tokens_details': None, 'prompt_time': 0.010654922, 'prompt_tokens_details': None, 'queue_time': 0.056488868, 'total_time': 0.056180621}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd6c3-a574-7ff0-96ad-c64240fa496d-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Bangalore'}, 'id': 'vmrcvh9ph', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 220, 'output_tokens': 15, 'total_tokens': 235}),
 ToolMessage(conte

In [6]:
model_with_tools

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000026E618AED10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026E618AEC20>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get the weather at a location', 'parameters': {'properties': {'location': {'type': 'string'}}, 'required': ['location'], 'type': 'object'}}}]}, config={}, config_factories=[])